# Scrapping with BeautifulSoup 

Scrapper des données sur le Web à l'aide des librairies **BeautifulSoup** et **Requests**

In [9]:
import requests
from bs4 import BeautifulSoup

## Requests, récuprer le code d'un site web

**requests** est un module python qui sert à faire des requêtes sur le web via les protocoles HTTP  

Avec se module nous allons pouvoir récuperer le code HTML d'un site web via son URL

Quelques méthodes de *`requests`*:

>* *`.get('url')`* LA commande qui permet d'aller chercher via une URL le format brut HTML sous forme d'un objet "HTTPResponse" du module urllib3.  Possible argument *`auth=('username','password')`* si il y a besoins d'une authentification  
>* *`.post`* permet d'envoyer des données vers le siteweb comme un *username* et un *psswd*  à travers un *`dict`*, ou même des img..
>* *`sess = requests.Session()`*; puis  Pour effectuer plusieurs requêtes sur un site, qui permet de conserver les identification et les cookies pendant l'exécution de toutes les requêtes. L'objet *`Session`* ici *`sess`* possède toute les méthodes de `requests` dont *`.get`* par exemple.

On peut lire la réponse de *`.get`* avec:
>* *`r.text`* Retourne le contenu en unicode
>* *`r.content`* Retourne le contenu en bytes
>* *`r.json`* Retourne le contenu sous forme json
>* *`r.headers`* Retourne le headers sous forme de dictionnaire 

In [10]:
html = requests.get('https://en.wikipedia.org/wiki/Main_Page')

In [11]:
html.headers

{'Date': 'Fri, 17 Apr 2020 16:21:12 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Server': 'mw1322.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Fri, 17 Apr 2020 16:21:12 GMT', 'Content-Encoding': 'gzip', 'Age': '19', 'X-Cache': 'cp3052 miss, cp3064 hit/170', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=17-Apr-2020;Path=/;HttpOnly;secure;Expires=Tue, 19 May 2020 12:00:00 GMT, WMF-Last-Access-Global=17-Apr-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Tue, 19 May 2020 12:00:00 GMT, GeoIP=FR:IDF:Paris:48.85:2.35:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '176.158.40.138', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalid

## BeautifulSoup pour analyser le HTML  
Començons par créer un Objet BeautfulSoup avec la réponse récupérée par *`.get()`* avec la méthode *`.content`* de *`requests`*, pour pouvoir ensuite utiliser toutes les méthodes d'analyse de HTML de BS.  
On lui passe en arg *`html.parser`* pour lui signifier que c'est bien du html.

In [14]:
soup = BeautifulSoup(html.content, "html.parser")

<img src="img.png"/>

## références
**Requests**
- http://fr.python-requests.org/en/latest/user/quickstart.html#requetes-post-avancees
- https://requests-fr.readthedocs.io/en/latest/user/advanced.html
- http://dridk.me/python-requests.html

**BeautifulSoup**
- http://jhroy.ca/uqam/edm5240/BeautifulSoup-DocAbregee.pdf